In [559]:
import math
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from question_set import 


In [560]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aakanksha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aakanksha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [561]:
#Set of all stopwords
stopwords=set(stopwords.words('english'))
stemmer=PorterStemmer()

In [562]:
q_corpus=pd.read_csv('questions.csv')
test_q=input("Enter a question :")

Enter a question :when was apollo 13 launched


In [563]:
#Query preprocessing-Tokenization,removing stopwords and stemming
def process_query(query):
    tokens=word_tokenize(query)
    tokens=[w.lower() for w in tokens]
    tokens=[w for w in tokens if w not in stopwords]
    tokens=[stemmer.stem(w) for w in tokens]
    tokens=[w for w in tokens if w!='?']
    return tokens

In [564]:
#Calculate term frequency of the query doc
def tf_value(q_entry):
    tf_mat={}
    terms=[]
    terms=process_query(q_entry)
    for t in terms:
        if t in tf_mat:
            tf_mat[t]+=1
        else:
            tf_mat[t]=1
    return tf_mat
    
    

In [565]:
#test_q='Where was Beyonce Knowles born?'
tf_q=tf_value(test_q)
for t in tf_q:
    print("%s : %d "%(t,tf_q[t]))


apollo : 1 
13 : 1 
launch : 1 


In [566]:
#Finding the idf of each term

def idf_value(q_entry,q_data):
    
    org_q=process_query(q_entry)#process the query
    
    idf_dict={}
    #Make dictionary containing doc freq of each term in query
    for t in org_q:
        idf_dict[t]=0
   
    #Check for every question_doc if it contains the term
    questions=q_data['Question']
    for q in questions:
        proc_q=process_query(q)
        for t in org_q:
            if t in proc_q:
                idf_dict[t]+=1
    for t in idf_dict:
        if idf_dict[t]>0:
            idf_dict[t]=math.log10(len(q_data)/float(idf_dict[t]))
        
    
    return idf_dict
             
        


In [567]:
#For the test question tf-idf vector
def test_tf_idf_vector(test_q,q_corpus):
    
    tf_q=tf_value(test_q)
    idf_q=idf_value(test_q,q_corpus)
    
    tf_idf_wt={}
    tf_wt={}
    for t in tf_q:
        tf_wt[t]=1+math.log10(tf_q[t])
        tf_idf_wt[t]=tf_wt[t]*idf_q[t]
    
    norm=0
    for t in tf_idf_wt:
        norm=norm+pow(tf_idf_wt[t],2)
    norm=math.sqrt(norm)
    
    for t in tf_idf_wt:
        tf_idf_wt[t]=tf_idf_wt[t]/(float(norm))
        
    return tf_idf_wt

        

In [568]:
#Calculating the tf-idf weight matrix for all the documents
def doc_tf_idf_vector(q_corpus):
    tf_docs={}
    
    for i in range(len(q_corpus)):
        tf_docs[q_corpus['QuestionID'][i]]=tf_value(q_corpus['Question'][i])
        
   # tf_wt={}
    tf_idf_wt={}
    
    for i in range(len(q_corpus)):
        tf_wt={}
        tf=tf_docs[q_corpus['QuestionID'][i]]
        idf=idf_value(q_corpus['Question'][i],q_corpus)
        
        #question=q_corpus['Question'][i]
        #question=process_query(question)
        for t in tf:
            tf_wt[t]=1+math.log10(tf[t])
            tf_wt[t]=tf_wt[t]*idf[t]
            
        norm=0
        for t in tf_wt:
            norm=norm+pow(tf_wt[t],2)
        norm=math.sqrt(norm)
        for t in tf_wt:
            tf_wt[t]=tf_wt[t]/(float(norm))
            
        tf_idf_wt[q_corpus['QuestionID'][i]]=tf_wt
    return tf_idf_wt
            
            
                

In [569]:
#Calculate the cosine similarity

def cosine_sim(q_corpus,test_q):
    
    query_vect=test_tf_idf_vector(test_q,q_corpus)
    question_doc_vector=doc_tf_idf_vector(q_corpus)
    
    max_cosine=0
    max_cos_qid=0
    
    for q in question_doc_vector:
        q_vector=question_doc_vector[q]
        
        #common=set(q_vector) & set(query_vect)
        
        cosine=0
        for w in q_vector:
            if w in query_vector:
                cosine=cosine+q_vector[w]*query_vector[w]
        #print(cosine)
        if cosine>max_cosine:
            max_cosine=cosine
            max_cos_qid=q
    return max_cos_qid
      
    
    

In [570]:
qid=cosine_sim(q_corpus,test_q)
print(qid)
#query_vector=test_tf_idf_vector(test_q,q_corpus)
#doc_vector=doc_tf_idf_vector(q_corpus)

#print(query_vector)
#print(doc_vector)
#tf_q=tf_value(test_q)
#for q in tf_q:
#    print("%s : %d "%(q,tf_q[q]))
#idf_q=idf_value(test_q,q_corpus)
#for q in idf_q:
#    print("%s : %d "%(q,idf_q[q]) )

#q_set=q_corpus.head()
#qid=cosine_sim(q_set,test_q)
#print(qid)
#q_corpus['Question']
#q_corpus['QuestionID'][0]

Q5
